In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
# Load the data
df = pd.read_csv('/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Train.csv')
X = np.array('/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Train/Train/'+df['id_code']+'.jpg')
y = np.array(df['diagnosis'])
y = y.astype(str)

In [ ]:
y

In [ ]:
df_new = pd.DataFrame({"path": X,
                "diagnosis": y,})

print(df_new['path'][0])
print(df_new['diagnosis'][0])

In [ ]:
# Create a data generator for training and testing data
train_datagen = image.ImageDataGenerator(
                rescale=1./255,
                validation_split=0.2)

test_datagen = image.ImageDataGenerator(
                rescale = 1.0/255,
                validation_split=0.2)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_new,
    x_col='path',
    y_col='diagnosis',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_new,
    x_col='path',
    y_col='diagnosis',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

In [ ]:
# Load the pre-trained ResNet50 model
model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the weights of the pre-trained layers
for layer in model.layers:
    layer.trainable = False

# Add new classification layers
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(5, activation='softmax')(x)

# Create the model
model = tf.keras.models.Model(inputs=model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

In [ ]:
img = cv2.imread('/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Train/Train/Train_1.jpg')
img = cv2.resize(img, (150, 150))
img = np.expand_dims(img, axis=0)
rescaled_img = cv2.normalize(img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
pred = model.predict(rescaled_img)

pred

In [ ]:
path_test = '/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Train/Train/Train_10.jpg'
img = image.load_img(path_test, target_size=(150, 150))
img_arr = image.img_to_array(img)
img_arr = np.expand_dims(img_arr, axis=0)
# img_arr /= 255.0
pred = model.predict(img_arr)
pred_class = np.argmax(pred, axis=1)


print(pred)
print(pred_class[0])

# # Submission

In [ ]:
# Load the data
df_sub = pd.read_csv('/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Test.csv')
X_sub = np.array('/kaggle/input/diabetic-retinopathy-classification-for-superai-3/Test/Test/'+df_sub['id_code']+'.jpg')

df_Sub = pd.DataFrame({"path": X_sub})

df_Sub['path'][1]

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
predictions = []
for img_name in df_Sub['path']:
    img = image.load_img(img_name, target_size=(150, 150))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
#     img_arr /= 255.0
    pred = model.predict(img_arr)
    pred_class = np.argmax(pred, axis=1)
    predictions.append(pred_class[0])

# Print the prediction results
print("Prediction results:", predictions)

In [ ]:
Sample_Submission = pd.DataFrame({"id_code": df_sub['id_code'],
                                 "diagnosis": predictions})

Sample_Submission.to_csv("Submission.csv", index=False)




In [ ]:
Sample_Submission